In [3]:
%%file submissions/starting_kit/feature_extractor.py
# -*- coding: utf-8 -*-

from __future__ import unicode_literals

from sklearn.feature_extraction.text import TfidfVectorizer
import string
import unicodedata
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.utils.validation import check_is_fitted
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler
from sklearn.decomposition import LatentDirichletAllocation

# def document_preprocessor(doc):
#     # TODO: is there a way to avoid these encode/decode calls?
#     try:
#         doc = unicode(doc, 'utf-8')
#     except NameError:  # unicode is a default on python 3
#         pass
#     doc = unicodedata.normalize('NFD', doc)
#     doc = doc.encode('ascii', 'ignore')
#     doc = doc.decode("utf-8")
#     return str(doc)

def token_processor(tokens):
    stemmer = SnowballStemmer('english')
    for token in tokens:
        yield stemmer.stem(token)

class FeatureExtractor(TfidfVectorizer):
    """Convert a collection of raw docs to a matrix of TF-IDF features. """

    def __init__(self):
        # see ``TfidfVectorizer`` documentation for other feature
        # extraction parameters.
        super(FeatureExtractor, self).__init__(
                analyzer='word',ngram_range = (1,2),
                stop_words='english', decode_error='replace',
                strip_accents='unicode',
                max_df = 1.0, min_df = 0.0001)
        self.lda = LatentDirichletAllocation(n_components=7, max_iter=20,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=None)

    def fit(self, X_df, y=None):
        """Learn a vocabulary dictionary of all tokens in the raw documents.

        Parameters
        ----------
        X_df : pandas.DataFrame
            a DataFrame, where the text data is stored in the ``statement``
            column.
        """
        X_tf = super(FeatureExtractor, self).fit_transform(X_df.statement)
        self.lda.fit(X_tf)
        return self

    def fit_transform(self, X_df, y=None):
        
        super(FeatureExtractor, self).fit(X_df)
        
        X_tf = self.transform(X_df)
        return self.lda.transform(X_tf)

    def transform(self, X_df):
        X = super(FeatureExtractor, self).transform(X_df.statement)
        return self.lda.transform(X)
    
    def build_tokenizer(self):
        """
        Internal function, needed to plug-in the token processor, cf.
        http://scikit-learn.org/stable/modules/feature_extraction.html#customizing-the-vectorizer-classes
        """
        tokenize = super(FeatureExtractor, self).build_tokenizer()
        return lambda doc: list(token_processor(tokenize(doc)))




Overwriting submissions/starting_kit/feature_extractor.py


In [4]:
!ramp_test_submission --quick-test

Testing Fake news: classify statements of public figures
Reading train and test files from ./data ...
Reading cv ...
Training ./submissions/starting_kit ...
CV fold 0
	score   sacc    acc  tfacc
	train  0.767  1.000  0.842
	valid  0.329  0.182  0.364
	test   0.419  0.240  0.631
CV fold 1
	score   sacc   acc  tfacc
	train  0.767  1.00  0.839
	valid  0.274  0.00  0.502
	test   0.415  0.21  0.604
CV fold 2
	score   sacc    acc  tfacc
	train  0.744  0.988  0.826
	valid  0.409  0.118  0.656
	test   0.414  0.230  0.621
CV fold 3
	score   sacc    acc  tfacc
	train  0.780  0.989  0.862
	valid  0.474  0.444  0.658
	test   0.408  0.220  0.626
CV fold 4
	score   sacc    acc  tfacc
	train  0.754  0.989  0.840
	valid  0.391  0.091  0.585
	test   0.401  0.240  0.596
CV fold 5
	score   sacc    acc  tfacc
	train  0.771  1.000  0.838
	valid  0.731  0.556  0.907
	test   0.414  0.230  0.621
CV fold 6
	score   sacc    acc  tfacc
	train  0.756  0.989  0.836
	valid  0.324  0.000  0.600
	test   0.386  0.220 